1. Объясните,  как  компьютерное  зрение  интерпретирует  цвет,  и  перечислите 
ключевые различия между моделями цвета RGB, CIE XYZ, и CIELAB. Как эти 
модели используются в обработке изображений, и какие их свойства делают их подходящими для конкретных приложений в области компьютерного зрения? 

2.  Напишите код на Python для реализации стохастического градиентного спуска и 
протестируйте его на наборе данных для линейной регрессии. 
 

| Характеристика  | RGB | YUV | HSV | CMYK | CIE XYZ | CIELAB |
|---------------|-----|-----|-----|------|--------|--------|
| **Тип модели** | Аппаратная, аддитивная | Кодировочная, субтрактивная | Перцепционная | Субтрактивная | Перцепционная, основана на зрении человека | Перцепционная, унифицированная |
| **Компоненты** | (R, G, B) – интенсивности каналов | (Y – яркость, U, V – цветоразность) | (H – тон, S – насыщенность, V – значение) | (C – циан, M – маджента, Y – желтый, K – черный) | (X, Y, Z) – триколорные стимулы | (L* – яркость, a* – красно-зеленый баланс, b* – сине-желтый баланс) |
| **Формат данных** | 8-битный (0-255) или плавающая точка (0-1) | 8-битный (0-255) или плавающая точка | Угловые градусы (H: 0-360°), проценты (S, V: 0-100%) | Проценты (0-100%) | Плавающая точка | Плавающая точка |
| **Линейность** | Линейное смешение цветов | Нелинейное, учитывает яркость | Нелинейное | Линейное для печати | Основана на измерениях восприятия | Нелинейно приближает восприятие человека |
| **Используется для** | Цифровые дисплеи, обработка изображений | Видеокодирование, сжатие, передача | Цветокоррекция, сегментация | Полиграфия, печать | Цветовой анализ, преобразования | Перцепционно равномерное представление, цветокоррекция |
| **Недостатки** | Не соответствует человеческому восприятию | Потери информации при сжатии | Неинтуитивное преобразование | Не подходит для экранов | Неудобен для восприятия человеком | Сложные вычисления |

---
| Тип модели           | Описание                                                                                 | Примеры применения     | Принцип работы                                              |
|----------------------|------------------------------------------------------------------------------------------|------------------------|------------------------------------------------------------|
| **Аддитивная (Additive)** | Основана на сложении цветов света. Чем больше света, тем ярче цвет.                   | RGB (дисплеи, проекторы) | Красный + Зеленый = Желтый, Красный + Синий = Пурпурный    |
| **Субтрактивная (Subtractive)** | Основана на поглощении света (смешении пигментов, красок). Чем больше цветов, тем темнее цвет. | CMYK (печать, полиграфия) | Желтый + Пурпурный = Красный, Желтый + Голубой = Зеленый   |
| **Перцепционная (Perceptual)** | Ориентирована на восприятие цвета человеком, учитывает особенности зрения.           | CIE XYZ, CIELAB         | Преобразование в пространства, которые воспринимаются человеком как равномерные |
| **Аппаратная (Hardware)** | Модели, использующие физические устройства для формирования цветов.                   | RGB (дисплеи)          | Прямое использование световых элементов (пикселей) для отображения цвета |

In [21]:
import numpy as np
import matplotlib.pyplot as plt
import os
from skimage.io import imread
from skimage.transform import resize

# Путь к папке с изображениями (поменяйте на свой путь)
image_folder_path = "../datasets/b_1"

# Функция для загрузки и предварительной обработки изображений
def load_images(image_folder_path, target_size=(100, 100)):
    image_files = [f for f in os.listdir(image_folder_path) if f.endswith(('.jpg', '.png'))]
    images = []
    
    for file in image_files:
        img = imread(os.path.join(image_folder_path, file))
        img_resized = resize(img, target_size)  # Уменьшаем изображения для ускорения
        images.append(img_resized)
        
    return np.array(images)

# Загрузка изображений
images = load_images(image_folder_path)

# Проверка формы загруженных изображений
print(f"Форма загруженных изображений: {images.shape}")

# Убедимся, что все изображения имеют одинаковую форму
if len(images.shape) == 4 and images.shape[3] == 3:
    # Преобразуем изображения в одномерные массивы (каждое изображение будет одной строкой)
    X = images.reshape(images.shape[0], -1)  # Каждое изображение в строку
    X = X / 255.0  # Нормализуем значения пикселей
    
    # Для простоты, используем только один канал (например, красный)
    y = X[:, 0]  # Предсказываем значение первого канала (красный)

    # Добавляем единичный столбец для смещения
    X_b = np.c_[np.ones((X.shape[0], 1)), X]

    # Функция для стохастического градиентного спуска
    def sgd(X, y, theta, learning_rate, epochs):
        m = len(X)
        for epoch in range(epochs):
            for i in range(m):
                # Выбор случайного примера для градиентного шага
                random_index = np.random.randint(m)
                xi = X[random_index:random_index+1]
                yi = y[random_index:random_index+1]
                
                # Предсказание и ошибка
                prediction = xi.dot(theta)
                error = prediction - yi
                
                # Градиент
                gradient = xi.T.dot(error)
                
                # Обновление параметров
                theta = theta - learning_rate * gradient
                
            # Печать ошибки каждые 100 эпох для наблюдения процесса
            if epoch % 100 == 0:
                loss = np.mean((X.dot(theta) - y) ** 2)
                print(f"Epoch {epoch}, Loss: {loss:.4f}")
        
        return theta

    # Инициализация параметров модели
    theta = np.random.randn(X_b.shape[1])  # для каждого канала + смещение
    learning_rate = 0.001  # Уменьшаем learning rate для стабильности
    epochs = 1000

    # Запуск алгоритма
    theta = sgd(X_b, y, theta, learning_rate, epochs)

    print(f"Обученные параметры: {theta}")

    # Предсказание на основе обученных параметров
    y_pred = X_b.dot(theta)

    # Визуализация результата
    plt.figure(figsize=(12, 6))

    # Оригинальные изображения
    plt.subplot(1, 2, 1)
    plt.imshow(images[0])
    plt.title(f"Оригинальное изображение")
    plt.axis('off')

    # Предсказанное изображение (для первого изображения)
    y_pred_image = y_pred[0].reshape(images[0].shape[0], images[0].shape[1])  # Восстанавливаем в исходный размер
    plt.subplot(1, 2, 2)
    plt.imshow(y_pred_image, cmap='gray')
    plt.title(f"Предсказанное изображение (красный канал)")
    plt.axis('off')

    plt.show()

else:
    print("Ошибка: Загруженные изображения не имеют ожидаемой формы (n_samples, height, width, channels).")

Форма загруженных изображений: (0,)
Ошибка: Загруженные изображения не имеют ожидаемой формы (n_samples, height, width, channels).
